<a href="https://colab.research.google.com/github/Tiabet/Complete_story/blob/master/KoreanSentenceEval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BLEURT, BERTSCORE, ROUGE-1 구현

In [ ]:
pip install git+https://github.com/google-research/bleurt.git

In [ ]:
pip install evaluate

In [68]:
pip install rouge

In [79]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (65/65), done.


In [80]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
!bash install_mecab-ko_on_colab_light_220429.sh

In [109]:
!git clone https://github.com/google-research/bleurt.git
!cd bleurt
!pip install .

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116
Receiving objects: 100% (134/134), 31.28 MiB | 22.68 MiB/s, done.
Resolving deltas: 100% (49/49), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
import evaluate

evaluate.list_evaluation_modules()

사용가능한 평가지표에 bleurt, rouge-1, bertscore 가 모두 있는 것으로 확인된다.
한국어, 영어 그리고 여타 언어들 모두 상관 없이 평가 기준은 동일할 것으로 보인다.

In [110]:
import evaluate

# Define the candidate predictions and reference sentences
predictions = ["안녕하신가요?", "철수는 집을 좋아한다."]
references = ['안녕하세요', '철수는 집에 가는 것을 좋아한다.']

# Load the BLEU evaluation metric
bleurt = bleurt = evaluate.load("bleurt", module_type="metric", checkpoint="bleurt-base-128")

# Compute the BLEU score
results = bleurt.compute(predictions=predictions, references=references)

# Print the results
print(results)

AttributeError: ignored

In [20]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["안녕하세요", "철수는 집을 좋아한다.", "좋아하는 음식이 뭐에요?"]
references = ['안녕하세요', '철수는 집에 가는 것을 좋아한다.', '선호하는 음식이 있으세요?']
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")


In [21]:
results

{'precision': [1.0, 0.9832342267036438, 0.9310119152069092],
 'recall': [1.0, 0.9639695882797241, 0.8979893922805786],
 'f1': [1.0, 0.9735065698623657, 0.9142025709152222],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.33.1)'}

확인해보니 evaluate의 rouge는 아직 한국어 패키지를 지원하지 않았음. rouge-1 이라는 개념 자체가 그냥 1-gram 씩, 즉 한 단어씩 비교해가면서 같은지 다른지를 체크하는 것. 때문에 형태소 분석을 진행해줘야 하고, 이를 위해서 Mecab을 많이들 사용하는 것 같음

In [82]:
from rouge import Rouge

rouge = Rouge()
predictions = ["안녕하세요", "철수는 집을 좋아한다.", "좋아하는 음식이 뭐에요?"]
references = ['안녕하세요', '철수는 집에 가는 것을 좋아한다.', '선호하는 음식이 있으세요?']
rouge.get_scores(predictions, references, avg=True)

{'rouge-1': {'r': 0.5777777777777777,
  'p': 0.6666666666666666,
  'f': 0.6111111062152778},
 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.5777777777777777,
  'p': 0.6666666666666666,
  'f': 0.6111111062152778}}

In [106]:
from konlpy.tag import Mecab
mecab = Mecab()
predictions = ["안녕하세요", "철수는 집을 좋아한다.", "좋아하는 음식이 뭐에요?"]
references = ['안녕하세요', '철수는 집에 가는 것을 좋아한다.', '선호하는 음식이 있으세요?']
for idx, s in enumerate(references):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        references[idx] = tokenized
for idx, s in enumerate(predictions):
        tokenized = []
        for mor in mecab.morphs(s):
            tokenized.append(mor)
        predictions[idx] = tokenized

for idx,sentence in enumerate(predictions):
  together = " ".join(sentence)
  predictions[idx] = together
for idx,sentence in enumerate(references):
  together = " ".join(sentence)
  references[idx] = together
rouge.get_scores(predictions, references, avg=True)

{'rouge-1': {'r': 0.75, 'p': 0.8333333333333334, 'f': 0.7749287700381086},
 'rouge-2': {'r': 0.6011904761904762,
  'p': 0.6944444444444445,
  'f': 0.624999995211227},
 'rouge-l': {'r': 0.75, 'p': 0.8333333333333334, 'f': 0.7749287700381086}}